In [ ]:
!pip install yfinance

     |████████████████████████████████| 6.3 MB 5.3 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.64-py2.py3-none-any.whl size=24109 sha256=7bfa1dcce65535d231573d2ae49b9717a6fcc1c0d9622d3bddc7153f5da56f3b
  Stored in directory: /root/.cache/pip/wheels/86/fe/9b/a4d3d78796b699e37065e5b6c27b75cff448ddb8b24943c288
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
import pandas as pd
#url="https://raw.githubusercontent.com/jfrausig/test/master/ticker_3.csv"
#df=pd.read_csv(url, error_bad_lines=False)
data=pd.read_csv("http://pvexpect.com/IND310_2021_ESG/portfolio_3.csv", delimiter=";")

aktier=[]
for index, row in data.iterrows():
    d=row.to_dict()
    aktier.append(d)
    #print(d)
aktier=pd.DataFrame(aktier)

aktier

,Sector,industry,shortName222,country,website,ticker,longBusinessSummary,Virksomhed,Sak,Unnamed: 9,Unnamed: 10
0,Consumer Cyclical,Home Improvement Retail,Fiskars Corporation,Finland,http://www.fiskarsgroup.com,FSKRS.HE,Fiskars Oyj Abp manufactures and markets consu...,Fiskars Oyj Abp,Bærekraft,NaN,NaN
1,Consumer Cyclical,Residential Construction,Bonava AB ser. B,Sweden,http://www.bonava.com,BONAV-B.ST,Bonava AB (publ) develops and sells residentia...,Bonava AB (publ),Bærekraft,NaN,NaN
2,Consumer Cyclical,Restaurants,MARLEYSPN CDI1000:1,Germany,http://www.marleyspoon.de,MMM.AX,Marley Spoon AG provides subscription-based we...,Marley Spoon AG,Mat industri,NaN,https://ir.marleyspoon.com/investor-centre/?pa...
3,Consumer Defensive,Beverages—Brewers,Carlsberg B A/S,Denmark,http://www.carlsberggroup.com,CARL-B.CO,Carlsberg A/S engages in the production and sa...,Carlsberg A/S,Mat industri,NaN,https://www.carlsberggroup.com/investor-relati...
4,Consumer Defensive,Beverages—Brewers,Royal UNIBREW A/S,Denmark,http://www.royalunibrew.com,RBREW.CO,"Royal Unibrew A/S produces, markets, sells, an...",Royal Unibrew A/S,Mat industri,NaN,https://investor.royalunibrew.com/
...,...,...,...,...,...,...,...,...,...,...,...
127,Utilities,Utilities—Independent Power Producers,VERBIO VER.BIOENERGIE ON,Germany,http://www.verbio.de,VBK.DE,VERBIO Vereinigte BioEnergie AG produces and s...,VERBIO Vereinigte BioEnergie AG,Bærekraft,NaN,NaN
128,Utilities,Utilities—Renewable,ENCAVIS AG INH. O.N.,Germany,http://www.encavis.com,ECV.DE,"Encavis AG, an independent power producer, acq...",Encavis AG,Bærekraft,NaN,NaN
129,Utilities,Utilities—Renewable,ENERGIEKONTOR O.N.,Germany,http://www.energiekontor.de,EKT.DE,"Energiekontor AG, a project developer, engages...",Energiekontor AG,Bærekraft,NaN,NaN
130,Utilities,Utilities—Renewable,Fortum Corporation,Finland,http://www.fortum.com,FORTUM.HE,"Fortum Oyj, together with its subsidiaries, en...",Fortum Oyj,Bærekraft,NaN,NaN


In [ ]:
aktier = aktier.assign(**{'aktie': ""})

aktier.index=aktier.ticker
for i, ticker in enumerate(aktier[aktier['Virksomhed']. notna()].ticker):

  name=aktier['Virksomhed'][ticker]
  aktier['Virksomhed'][ticker]=str(name).replace(".","").replace("-","").replace("(","").replace(")","").replace("/","")

aktier #[aktier['shortName']. notna()]

,Sector,industry,shortName222,country,website,ticker,longBusinessSummary,Virksomhed,Sak,Unnamed: 9,Unnamed: 10,aktie
ticker,,,,,,,,,,,,
FSKRS.HE,Consumer Cyclical,Home Improvement Retail,Fiskars Corporation,Finland,http://www.fiskarsgroup.com,FSKRS.HE,Fiskars Oyj Abp manufactures and markets consu...,Fiskars Oyj Abp,Bærekraft,NaN,NaN,
BONAV-B.ST,Consumer Cyclical,Residential Construction,Bonava AB ser. B,Sweden,http://www.bonava.com,BONAV-B.ST,Bonava AB (publ) develops and sells residentia...,Bonava AB publ,Bærekraft,NaN,NaN,
MMM.AX,Consumer Cyclical,Restaurants,MARLEYSPN CDI1000:1,Germany,http://www.marleyspoon.de,MMM.AX,Marley Spoon AG provides subscription-based we...,Marley Spoon AG,Mat industri,NaN,https://ir.marleyspoon.com/investor-centre/?pa...,
CARL-B.CO,Consumer Defensive,Beverages—Brewers,Carlsberg B A/S,Denmark,http://www.carlsberggroup.com,CARL-B.CO,Carlsberg A/S engages in the production and sa...,Carlsberg AS,Mat industri,NaN,https://www.carlsberggroup.com/investor-relati...,
RBREW.CO,Consumer Defensive,Beverages—Brewers,Royal UNIBREW A/S,Denmark,http://www.royalunibrew.com,RBREW.CO,"Royal Unibrew A/S produces, markets, sells, an...",Royal Unibrew AS,Mat industri,NaN,https://investor.royalunibrew.com/,
...,...,...,...,...,...,...,...,...,...,...,...,...
VBK.DE,Utilities,Utilities—Independent Power Producers,VERBIO VER.BIOENERGIE ON,Germany,http://www.verbio.de,VBK.DE,VERBIO Vereinigte BioEnergie AG produces and s...,VERBIO Vereinigte BioEnergie AG,Bærekraft,NaN,NaN,
ECV.DE,Utilities,Utilities—Renewable,ENCAVIS AG INH. O.N.,Germany,http://www.encavis.com,ECV.DE,"Encavis AG, an independent power producer, acq...",Encavis AG,Bærekraft,NaN,NaN,
EKT.DE,Utilities,Utilities—Renewable,ENERGIEKONTOR O.N.,Germany,http://www.energiekontor.de,EKT.DE,"Energiekontor AG, a project developer, engages...",Energiekontor AG,Bærekraft,NaN,NaN,


In [ ]:
#@title Hent aktier
import yfinance as yf

def stockDownload(symbol):

  stock = yf.Ticker(symbol)

  hist = stock.history(period="2y")

  return hist


In [ ]:
import bs4 as bs
import pickle
import requests
import os
import pandas as pd

def save_sp500_tickers():

    tickers = []
    for aktie in aktier:
        ticker = aktie
        tickers.append(ticker)
    with open("sp500tickers.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers

save_sp500_tickers()
def get_data_from_yahoo(reload_sp500=False):
    
    if reload_sp500:
        
        tickers = save_sp500_tickers()
    else:
        
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')



    #for i, n, in enumerate(aktier):
    for i in range(0,len(aktier)):
        #print(n)
        
        #print()
        aktie=aktier['Virksomhed'][i]
        ticker=aktier['ticker'][i]
        stock_name=str(aktier['ticker'][i])
        # just in case your connection breaks, we'd like to save our progress!
        if not os.path.exists('stock_dfs/{}.csv'.format(aktie)):
            
            df=stockDownload(stock_name)
 
            df.to_csv('stock_dfs/{}.csv'.format(aktie))
            print('Downloading {}'.format(aktie))
        else:
            print('Already have {}'.format(aktie))


get_data_from_yahoo()

- CLNK-B.ST: No data found, symbol may be delisted
- OSR.DE: No data found, symbol may be delisted


In [ ]:
import pandas as pd
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
        print(tickers)
    main_df = pd.DataFrame()

    #for count, ticker in enumerate(tickers):

    for i in range(0,len(aktier)):
        #print(n)
        
        #print()
        aktie=aktier['Virksomhed'][i]
        ticker=aktier['ticker'][i]
        print(aktie)
        #stock_name=str(aktier['Ticker'][i])

        ds=pd.DataFrame()
        print(ticker)
        df = pd.read_csv('stock_dfs/{}.csv'.format(aktie))
        df.set_index('Date', inplace=True)

        df.dropna(inplace=True)
        #df.index=df.Date
        #print(df.info())
        try:
          ds['Close']=df['Adj Close']
        except:
          #ds[ticker]=df['Close']
          print("ok")
        
        ds[aktie]=df['Close']
        ds.rename(columns={'Close': aktie}, inplace=True)
        print(ds)


        if main_df.empty:
            main_df = ds
        else:
            main_df = main_df.join(ds, how='outer')

        if i % 10 == 0:
            print(i)
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')


compile_data()

['Sector', 'industry', 'shortName222', 'country', 'website', 'ticker', 'longBusinessSummary', 'Virksomhed', 'Sak', 'Unnamed: 9', 'Unnamed: 10', 'aktie']
Fiskars Oyj Abp
FSKRS.HE
ok
            Fiskars Oyj Abp
Date                       
2019-10-21        11.576637
2019-10-22        11.392880
2019-10-23        11.374505
2019-10-24        11.245875
2019-10-25        11.374505
...                     ...
2021-10-15        21.200001
2021-10-18        21.450001
2021-10-19        21.100000
2021-10-20        21.150000
2021-10-21        20.900000

[503 rows x 1 columns]
0
Bonava AB publ
BONAV-B.ST
ok
            Bonava AB publ
Date                      
2019-10-21      104.978973
2019-10-22      102.761101
2019-10-23       91.486946
2019-10-24       86.912613
2019-10-25       87.975327
...                    ...
2021-10-15       84.349998
2021-10-18       85.449997
2021-10-19       86.500000
2021-10-20       86.500000
2021-10-21       86.599998

[503 rows x 1 columns]
Marley Spoon AG
MMM.AX
ok

In [ ]:
import pandas as pd
import numpy as np

def ohlc():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)
        print(tickers)

    main_df = pd.DataFrame()


    for i in range(0,len(aktier)):

        aktie=aktier['Virksomhed'][i]
        ticker=aktier['ticker'][i]
        stock_name=str(aktier['ticker'][i])

        ds=pd.DataFrame()

        df = pd.read_csv('stock_dfs/{}.csv'.format(aktie))
        df.Date=pd.to_datetime(df.Date)
        df.set_index('Date', inplace=True)

        try:
          #ds['Close']=df['Adj Close']
          df.rename(columns={'Adj Close': Close}, inplace=True)
        except:
          #ds[ticker]=df['Close']
          print("ok")

        df['Log_Ret']=np.log(df.Close/df['Close'].shift(1))
        df['Volatility']=df.Log_Ret.rolling(window=30).std()*np.sqrt(30)
        #df['date']=df.index

        data = df['Close'].resample('4D').ohlc()

        #try:
        #  df[['open', 'close', 'high', 'low']]=data[['open', 'close', 'high', 'low']]
        #except:
        #  print("not in index")


        df['Virksomhed']=aktie


        main_df=pd.concat([main_df,df])
    print(main_df)
    main_df.to_csv('sp500_ohlc.csv')
ohlc()



['Sector', 'industry', 'shortName222', 'country', 'website', 'ticker', 'longBusinessSummary', 'Virksomhed', 'Sak', 'Unnamed: 9', 'Unnamed: 10', 'aktie']
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
not in index
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
not in index
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
                  Open        High  ...       Virksomhed  Adj Close
Date                                ...                            
2019-10-21   11.466383   11.576637  ...  Fiskars Oyj Abp        NaN
2019-10-22   11.576637   11.576637  ...  Fiskars Oyj Abp        NaN
2019-10-23   11.411256   11.429631  ...  Fiskars Oyj Abp        NaN
2019-10-24   11.374505   11.429631  ...  Fiskars Oyj Abp        NaN
2019-10-25   11.3

In [ ]:
#Import Libraries
#https://towardsdatascience.com/python-risk-management-monte-carlo-simulations-7d41c891cb5
import numpy as np  
import pandas as pd  
import pandas_datareader as wb  
import matplotlib.pyplot as plt  
from scipy.stats import norm
%matplotlib inline

"""
with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

main_df = pd.DataFrame()
"""

df = pd.read_csv('sp500_joined_closes.csv')

print(df)
#for count, ticker in enumerate(tickers):
labels=[]

fordeling=pd.DataFrame()
start_fordeling=pd.DataFrame()
slut_fordeling=pd.DataFrame()

for i in range(0,len(aktier)):
  #print(n)
  
  #print()
  aktie=aktier['Virksomhed'][i]
  ticker=aktier['ticker'][i]
#for i, n, in enumerate(aktier):
  #ticker=n
  #stock_name=str(aktier[n])

  data=pd.DataFrame()

  #print(ticker)
  #Settings for Monte Carlo asset data, how long, and how many forecasts 
  #ticker = 'S92.DE' # ticker
  t_intervals = 30 # time steps forecasted into future
  iterations = 200 # amount of simulations
  #Acquiring data
  #data = pd.read_csv('XTZ_USD Huobi Historical Data.csv',index_col=0,usecols=['Date', 'Price'])
  #data=stockDownload(stock_name).drop(["Open","High","Low","Volume","Dividends","Stock Splits"], axis=1) #.dropna(inplace=True)
  #print(data)
  #data = data.rename(columns={"Price": ticker})
  df.index=df.Date
  try:
    data[aktie]=df[aktie].dropna()
  

    #print(data)
    #print(data[(data.index<'2020-12-04')]) #
    #data=data[(data.index<'2020-12-04')]
    #data=data[(data.index>'2020-11-15')]
    #print(data)
    #Preparing log returns from data
    log_returns = np.log(1 + data.pct_change())
    #Plot of asset historical closing price
    #data.plot(figsize=(10, 6));

    #Setting up drift and random component in relation to asset data
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5 * var)
    stdev = log_returns.std()
    daily_returns = np.exp(drift.values + stdev.values * norm.ppf(np.random.rand(t_intervals, iterations)))
    #Takes last data point as startpoint point for simulation
    S0 = data.iloc[-1]
    price_list = np.zeros_like(daily_returns)
    price_list[0] = S0
    #Applies Monte Carlo simulation in asset
    for t in range(1, t_intervals):
        price_list[t] = price_list[t - 1] * daily_returns[t]


    data_sell=[]
    data_buy=[]
    data_total=[]
    #print(S0)
    #for i in price_list[-1]:
    for i, t, in enumerate(price_list[-1]):
      data_total.append(t)
      if S0[aktie]>t:
        data_sell.append(t)
      if S0[aktie]<t:
        data_buy.append(t)


    
    
    #fordeling['dist']=(price_list[-1])
    #start_fordeling['start_fordeling']=(price_list[0])
    fordeling[aktie]=(price_list[-1].mean()-price_list[-1])/price_list[0].mean() #price_list[-1].std()
    start_fordeling[aktie]=(price_list[-1].mean()-price_list[0])/price_list[0].mean() #price_list[-1].std()
    
    labels.append(aktie)

  except:
    print("key error")
  #start_labels.append(ticker+"_start")

print(labels)

slut_fordeling=fordeling.median()
print(slut_fordeling)

#print(start_fordeling)
#brush = alt.selection(type='interval', encodings=['x'], name='sel') #opret areal markering



           Date  Fiskars Oyj Abp  ...  Fortum Oyj   Ørsted AS
0    2019-10-21        11.576637  ...   17.424643  606.527466
1    2019-10-22        11.392880  ...   17.814203  611.786438
2    2019-10-23        11.374505  ...   17.879129  612.955078
3    2019-10-24        11.245875  ...   17.530149  607.890930
4    2019-10-25        11.374505  ...   17.424643  613.539368
..          ...              ...  ...         ...         ...
510  2021-10-15        21.200001  ...   26.750000  887.799988
511  2021-10-18        21.450001  ...   26.680000  877.000000
512  2021-10-19        21.100000  ...   26.410000  890.000000
513  2021-10-20        21.150000  ...   27.290001  915.400024
514  2021-10-21        20.900000  ...   27.129999  914.599976

[515 rows x 135 columns]
key error
key error
['Fiskars Oyj Abp', 'Bonava AB publ', 'Marley Spoon AG', 'Carlsberg AS', 'Royal Unibrew AS', 'Cloetta AB publ', 'KWS SAAT SE & Co KGaA', 'Beiersdorf Aktiengesellschaft', 'Duni AB publ', 'Essity AB publ', 'Henke

In [ ]:
slut_fordeling

Fiskars Oyj Abp                   -0.001396
Bonava AB publ                     0.009499
Marley Spoon AG                    0.047315
Carlsberg AS                       0.003687
Royal Unibrew AS                   0.002039
                                     ...   
VERBIO Vereinigte BioEnergie AG    0.020411
Encavis AG                         0.007208
Energiekontor AG                   0.002581
Fortum Oyj                         0.002515
Ørsted AS                         -0.003531
Length: 130, dtype: float64

In [ ]:
#https://towardsdatascience.com/altair-plot-deconstruction-visualizing-the-correlation-structure-of-weather-data-38fb5668c5b1
import altair as alt
#def visualize_data():
df = pd.read_csv('sp500_joined_closes.csv')
#df_corr = df[['Cummins','PACCAR']].corr()
df_corr = df.corr()

df_corr.to_csv('sp500corr.csv')

alt.data_transformers.disable_max_rows()


cor_data = df_corr.stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'Virksomhed', 'level_1': 'aktie_2'}).sort_values(by="correlation",kind="quicksort")       # The stacking results in an index on the correlation values, we need the index as normal columns for Altair
#print(list(cor_data.variable))             
cor_data['Aktier'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal


aktie_navne=aktier #"https://raw.githubusercontent.com/jfrausig/test/master/ticker_3.csv"

industry=alt.binding_select(options=list(aktier.industry.unique()))
Aktie_selector = alt.selection_single(fields=['industry'],name='industry', bind=industry, #, clear=False,  #, 'variable2']
                                  init={'industry': aktier.industry[0]}) 

base =alt.Chart(cor_data).mark_bar().transform_lookup(
    lookup='aktie_2',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).transform_filter(
  #Aktie_selector
  'datum.industry==industry.industry' #||datum.sektor==sektor.sektor&&datum.aktie_2==datum.aktie_2', #&&datum.variable2==datum.aktie
).encode(

x=alt.X('aktie_2:O', axis=alt.Axis(orient='top',labelAngle=80,labelFontSize=8),sort=list('datum.Virksomhed')), #<,sort=list('datum.aktie')#,axis=alt.Axis(labelAngle=0) sort=list(cor_data.variable2),
y=alt.Y('Virksomhed:O',axis=alt.Axis(labelFontSize=8),sort=list('datum.Virksomhed')), #, sort=list(cor_data.aktie)
  
).properties(width=600,height=200)


var_sel_cor = alt.selection_single( fields=['Virksomhed'], clear=False,  #, 'variable2']
                                  init={'Virksomhed': 'Mowi'}) #, 'variable2': 'SalMar'


cor_plot = base.mark_rect().add_selection(var_sel_cor).transform_lookup(
    lookup='Virksomhed',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).encode(
    color='correlation:Q',
    #color=alt.condition(var_sel_cor, alt.value('pink'), 'correlation:Q')
    opacity = alt.condition(var_sel_cor, alt.value(1), alt.value(0.4)),
)

text_labels = base.mark_text(color='Aktier:N', size=8).encode(
    text='Aktier:N',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black'),
    #opacity = alt.condition(single_click, alt.value(1), alt.value(0.05)),
    
    )
    
)

alt.vconcat((cor_plot + text_labels).add_selection(Aktie_selector).transform_lookup(
    lookup='Virksomhed',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).transform_filter(
  #Aktie_selector
  'datum.industry==industry.industry' #||datum.sektor==sektor.sektor&&datum.aktie_2==datum.aktie_2', #&&datum.variable2==datum.aktie
))

alt.VConcatChart(...)

In [ ]:
chart_1

In [ ]:
#https://towardsdatascience.com/altair-plot-deconstruction-visualizing-the-correlation-structure-of-weather-data-38fb5668c5b1
import altair as alt
#def visualize_data():
df = pd.read_csv('sp500_joined_closes.csv')
#df_corr = df[['Cummins','PACCAR']].corr()
df_corr = df.corr()

df_corr.to_csv('sp500corr.csv')

alt.data_transformers.disable_max_rows()


cor_data = df_corr.stack().reset_index().rename(columns={0: 'correlation', 'level_0': 'Virksomhed', 'level_1': 'aktie_2'}).sort_values(by="correlation",kind="quicksort")       # The stacking results in an index on the correlation values, we need the index as normal columns for Altair
#print(list(cor_data.variable))             
cor_data['Aktier'] = cor_data['correlation'].map('{:.2f}'.format)  # Round to 2 decimal


aktie_navne=aktier #"https://raw.githubusercontent.com/jfrausig/test/master/ticker_3.csv"

industry=alt.binding_select(options=list(aktier.industry.unique()))
Aktie_selector = alt.selection_single(fields=['industry'],name='industry', bind=industry, #, clear=False,  #, 'variable2']
                                  init={'industry': aktier.industry[0]}) 

base =alt.Chart(cor_data).mark_bar().transform_lookup(
    lookup='aktie_2',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).transform_filter(
  #Aktie_selector
  'datum.industry==industry.industry' #||datum.sektor==sektor.sektor&&datum.aktie_2==datum.aktie_2', #&&datum.variable2==datum.aktie
).encode(

x=alt.X('aktie_2:O', axis=alt.Axis(orient='top',labelAngle=80,labelFontSize=8),sort=list('datum.Virksomhed')), #<,sort=list('datum.aktie')#,axis=alt.Axis(labelAngle=0) sort=list(cor_data.variable2),
y=alt.Y('Virksomhed:O',axis=alt.Axis(labelFontSize=8),sort=list('datum.Virksomhed')), #, sort=list(cor_data.aktie)
  
).properties(width=800,height=200)


var_sel_cor = alt.selection_multi( fields=['Virksomhed'], clear=False,  #, 'variable2']
                                   toggle=True) #, 'variable2': 'SalMar'


cor_plot = base.mark_rect().add_selection(var_sel_cor).transform_lookup(
    lookup='Virksomhed',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).encode(
    color='correlation:Q',
    #color=alt.condition(var_sel_cor, alt.value('pink'), 'correlation:Q')
    opacity = alt.condition(var_sel_cor, alt.value(1), alt.value(0.4)),
)

text_labels = base.mark_text(color='Aktier:N', size=8).encode(
    text='Aktier:N',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black'),
    #opacity = alt.condition(single_click, alt.value(1), alt.value(0.05)),
    
    )
    
)

cor=alt.vconcat((cor_plot + text_labels).add_selection(Aktie_selector).transform_lookup(
    lookup='Virksomhed',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['industry']) 
).transform_filter(
  #Aktie_selector
  'datum.industry==industry.industry' #||datum.sektor==sektor.sektor&&datum.aktie_2==datum.aktie_2', #&&datum.variable2==datum.aktie
))


source = pd.read_csv('sp500_ohlc.csv')# data.ohlc()

open_close_color = alt.condition("datum.Open <= datum.Close",
                                 alt.value("#06982d"),
                                 alt.value("#ae1325"))
ohlc_brush = alt.selection(type='interval', encodings=['x'])

ohlc_base = alt.Chart(source).encode(
    alt.X('Date:T',
          axis=alt.Axis(
              format='%m/%d',
              labelAngle=-45,
              title='Date in 2009'
          )
    ),
    color=open_close_color
).transform_filter(
    var_sel_cor
).properties(height=150, width=800)

ohlc_rule = ohlc_base.mark_rule().encode(
    alt.Y(
        'Low:Q',
        title='Price',
        scale=alt.Scale(zero=False),
    ),
    alt.Y2('High:Q')
).transform_filter(
    ohlc_brush
)

ohlc_bar = ohlc_base.mark_bar().encode(
    alt.Y('Open:Q'),
    alt.Y2('Close:Q')
).transform_filter(
    ohlc_brush
)

ohlc_chart = ohlc_base.transform_filter(
    var_sel_cor
).mark_line(color="blue").encode(
    alt.X('date:T'),
    alt.Y('Volatility:Q'),
    color='Virksomhed:N',
    tooltip=[alt.Tooltip('aktie:N'),alt.Tooltip('Volatility:Q', format='.2f')],
)




(cor & (ohlc_rule + ohlc_bar) & ohlc_chart).transform_lookup(
    lookup='Virksomhed',
    from_=alt.LookupData(aktie_navne, 'Virksomhed', ['sektor','aktie_nr']) 
).add_selection(
    var_sel_cor
).add_selection(ohlc_brush).properties(padding=alt.Padding({"left": 50, "top": 50, "right": 50, "bottom": 200})).save("dashboard.html")

